# TemporalDifference-Gammon

Während man mir dem Value-Player jetzt beliebige Aspekte des Spielbretts berechnen und bewerten lassen kann und aus den verschiedenen Funktionen neue kombinieren kann ist es doch sehr schwer und aufwändig von Hand alle Aspekte zu finden und zu schreiben.

Hier kommt nun TD-Gammon ins Spiel: Ein neuronales Netz das versucht eigenständig ohne Vorkenntnisse Backgammon zu lernen.

TD-Gammon besteht aus zwei Teilen: Der Spieler (der __TDGammonPlayer__) und Das neuronale Netz (das __Modell__)

## TD-GammonPlayer

Der TDGammonPlayer ist eine Subklasse vom ValuePlayer, da er zusätzlich noch ein (trainiertes) Modell benötigt um seine ValueFunction benutzen zu können

In [1]:
from Player import ValuePlayer

class TDGammonPlayer(ValuePlayer):
    
    def __init__(self, player, model):
        ValuePlayer.__init__(self, player, self.get_value)
        self.model = model
        
    def get_value(self, game, player):
        features = game.extractFeatures(player)
        v = self.model.get_output(features)
        v = 1 - v if self.player == game.players[0] else v
        return v
    
    def get_name(self):
        return "TD-GammonPlayer"

## TD-Gammon Modell

Basierend auf https://medium.com/jim-fleming/before-alphago-there-was-td-gammon-13deff866197

Das neuronale Netz für TD-Gammon ist recht einfach gehalten:

<ul>
    <li> 198 Input Units</li>
    <li> 40 Hidden Units</li>
    <li> 1 Output Units</li>
</ul>

Das neuronale Netz erzeugt als Ausgabe, ähnlich dem ValuePlayer, eine Zahl zwischen 0 und 1. Man kann das Modell also als komplizierte ValueFunction ansehen.

In [1]:
from Player import RandomPlayer
from FasterBackgammon import Game
import tensorflow as tf
import numpy as np
import random
import os

class TDGammonModel(object):
    
    def __init__(self, sess, restore=False):
        #Speicherort ermitteln
        self.checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'checkpoints/')
        
        #session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        #lambda, leider ohne b da dies ein Schlüsselwort ist
        lamda = tf.constant(0.7)

        #learning rate 
        alpha = tf.constant(0.1)

        #Für leichteres abändern der Netzwerkarchitektur
        layer_size_input = 198
        layer_size_hidden = 40
        layer_size_output = 1

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, layer_size_input], name='x')
        self.V_next = tf.placeholder('float', [1, layer_size_output], name='V_next')

        #Zwei fully-connected, dense Layer: Ein Input-Layer (198 Units) und dann ein Hidden-Layer (40 Units)
        #Beide werden mit der sigmoid Funktion aktiviert
        prev_y = tf.layers.dense(self.x, layer_size_hidden, activation= tf.sigmoid)
        self.V = tf.layers.dense(prev_y, layer_size_output, activation= tf.sigmoid)
        
        # delta = V_next - V
        delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        #Global_step wird bei jeden aufruf von sess.run um 1 erhöht
        global_step_op = self.global_step.assign_add(1)

        # get gradients of output V wrt trainable variables (weights and biases)
        tvars = tf.trainable_variables()
        grads = tf.gradients(self.V, tvars)

        #Alle Variablen werden angepasst und mittels eligebility traces wird er Wert
        #der einzelnen Gewichte angepasst um gute bzw. schlechte Entscheidungen zu reflektieren
        apply_gradients = []

        for grad, var in zip(grads, tvars):
            # e-> = lambda * e-> + <grad of output w.r.t weights>
            trace = tf.Variable(tf.zeros(grad.get_shape()), trainable=False, name='trace')
            trace_op = trace.assign((lamda * trace) + grad)

            # grad with trace = alpha * delta * e
            grad_trace = alpha * delta_op * trace_op

            grad_apply = var.assign_add(grad_trace)
            apply_gradients.append(grad_apply)
        
        #Den global_step mitzählen lassen
        apply_gradients.append(global_step_op)

        #Alle Gradientenoperationen in einer train Operation zusammenfassen
        self.train_op = tf.group(*apply_gradients, name='train')
        
        #Der Saver behält immer nur den neusten Checkpoint
        self.saver = tf.train.Saver(max_to_keep=1)

        #Variablen initialisieren 
        self.sess.run(tf.global_variables_initializer())

        #Modell wiederherstellen falls gewünscht
        if restore:
            self.restore()

    #Lädt den neusten (und auch einzigen) checkpoint 
    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print("Restoring checkpoint: {0}".format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    #Erzeugt einen Outpt für die gegebenen features x
    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    #Testet das Modell gegen den angegebenen enemyAgent
    def test(self, enemyPlayer=RandomPlayer('white'), games=100, debug=False):
        players = [TDGammonPlayer('black', self), enemyPlayer]
        
        winners = {'black':0, 'white':0}
        for i in range(games):
            game = Game()

            winner = game.play(players, debug=debug)
            winners[winner] += 1

            winners_total = sum(winners.values())
            print("[Game %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (i, \
                players[0].get_name(), players[0].player, \
                players[1].get_name(), players[1].player, \
                winners['black'], winners['white'], winners_total, \
                (winners['black'] / winners_total) * 100.0))
            
    def train(self, games, validation_interval, test_games=100):
        #Selbsttraining, Modell vs Modell
        players = [TDGammonPlayer('black', self), TDGammonPlayer('white', self)]

        for i in range(games):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if i != 0 and i % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(games = test_games)

            #Spiel initialisieren
            game = Game()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run(self.train_op, feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step = self.sess.run([
                self.train_op,
                self.global_step,
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (i, games, players[winner].player, game_step))
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step) 
        self.test(games = test_games)

### Los geht's!

Nun mit Modell und Player bereit, wie gut ist denn der Player?

In [50]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=False)
    model.test(games = 200)

[Game 0] TD-GammonPlayer (black) vs RandomPlayer (white) 0:1 of 1 games (0.00%)
[Game 1] TD-GammonPlayer (black) vs RandomPlayer (white) 1:1 of 2 games (50.00%)
[Game 2] TD-GammonPlayer (black) vs RandomPlayer (white) 1:2 of 3 games (33.33%)
[Game 3] TD-GammonPlayer (black) vs RandomPlayer (white) 1:3 of 4 games (25.00%)
[Game 4] TD-GammonPlayer (black) vs RandomPlayer (white) 1:4 of 5 games (20.00%)
[Game 5] TD-GammonPlayer (black) vs RandomPlayer (white) 2:4 of 6 games (33.33%)
[Game 6] TD-GammonPlayer (black) vs RandomPlayer (white) 2:5 of 7 games (28.57%)
[Game 7] TD-GammonPlayer (black) vs RandomPlayer (white) 3:5 of 8 games (37.50%)
[Game 8] TD-GammonPlayer (black) vs RandomPlayer (white) 3:6 of 9 games (33.33%)
[Game 9] TD-GammonPlayer (black) vs RandomPlayer (white) 3:7 of 10 games (30.00%)
[Game 10] TD-GammonPlayer (black) vs RandomPlayer (white) 3:8 of 11 games (27.27%)
[Game 11] TD-GammonPlayer (black) vs RandomPlayer (white) 4:8 of 12 games (33.33%)
[Game 12] TD-GammonPlaye

[Game 98] TD-GammonPlayer (black) vs RandomPlayer (white) 27:72 of 99 games (27.27%)
[Game 99] TD-GammonPlayer (black) vs RandomPlayer (white) 27:73 of 100 games (27.00%)
[Game 100] TD-GammonPlayer (black) vs RandomPlayer (white) 27:74 of 101 games (26.73%)
[Game 101] TD-GammonPlayer (black) vs RandomPlayer (white) 27:75 of 102 games (26.47%)
[Game 102] TD-GammonPlayer (black) vs RandomPlayer (white) 27:76 of 103 games (26.21%)
[Game 103] TD-GammonPlayer (black) vs RandomPlayer (white) 27:77 of 104 games (25.96%)
[Game 104] TD-GammonPlayer (black) vs RandomPlayer (white) 27:78 of 105 games (25.71%)
[Game 105] TD-GammonPlayer (black) vs RandomPlayer (white) 28:78 of 106 games (26.42%)
[Game 106] TD-GammonPlayer (black) vs RandomPlayer (white) 29:78 of 107 games (27.10%)
[Game 107] TD-GammonPlayer (black) vs RandomPlayer (white) 30:78 of 108 games (27.78%)
[Game 108] TD-GammonPlayer (black) vs RandomPlayer (white) 30:79 of 109 games (27.52%)
[Game 109] TD-GammonPlayer (black) vs RandomPl

[Game 192] TD-GammonPlayer (black) vs RandomPlayer (white) 54:139 of 193 games (27.98%)
[Game 193] TD-GammonPlayer (black) vs RandomPlayer (white) 54:140 of 194 games (27.84%)
[Game 194] TD-GammonPlayer (black) vs RandomPlayer (white) 54:141 of 195 games (27.69%)
[Game 195] TD-GammonPlayer (black) vs RandomPlayer (white) 54:142 of 196 games (27.55%)
[Game 196] TD-GammonPlayer (black) vs RandomPlayer (white) 55:142 of 197 games (27.92%)
[Game 197] TD-GammonPlayer (black) vs RandomPlayer (white) 55:143 of 198 games (27.78%)
[Game 198] TD-GammonPlayer (black) vs RandomPlayer (white) 55:144 of 199 games (27.64%)
[Game 199] TD-GammonPlayer (black) vs RandomPlayer (white) 55:145 of 200 games (27.50%)


Das ist alles? Sollte der nicht besser sein?

__Trainieren vergessen__

Das neuronale Netz weiß noch nichts über Backgammon und ohne Training spielt es nur zufällig, da es noch nicht weiß was gute bzw. schlechte Brettpositionen sind.

### Training

In [48]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=False)
    model.train(1000,1000)

Game 0/1000 (Winner: black) in 75 turns
Game 1/1000 (Winner: white) in 66 turns
Game 2/1000 (Winner: white) in 219 turns
Game 3/1000 (Winner: white) in 139 turns
Game 4/1000 (Winner: black) in 112 turns
Game 5/1000 (Winner: white) in 245 turns
Game 6/1000 (Winner: black) in 339 turns
Game 7/1000 (Winner: white) in 84 turns
Game 8/1000 (Winner: black) in 131 turns
Game 9/1000 (Winner: white) in 62 turns
Game 10/1000 (Winner: black) in 87 turns
Game 11/1000 (Winner: white) in 248 turns
Game 12/1000 (Winner: white) in 177 turns
Game 13/1000 (Winner: black) in 51 turns
Game 14/1000 (Winner: black) in 141 turns
Game 15/1000 (Winner: white) in 90 turns
Game 16/1000 (Winner: black) in 116 turns
Game 17/1000 (Winner: white) in 139 turns
Game 18/1000 (Winner: white) in 110 turns
Game 19/1000 (Winner: white) in 82 turns
Game 20/1000 (Winner: white) in 72 turns
Game 21/1000 (Winner: white) in 49 turns
Game 22/1000 (Winner: white) in 56 turns
Game 23/1000 (Winner: white) in 75 turns
Game 24/1000 (

Game 199/1000 (Winner: white) in 53 turns
Game 200/1000 (Winner: white) in 45 turns
Game 201/1000 (Winner: white) in 52 turns
Game 202/1000 (Winner: white) in 56 turns
Game 203/1000 (Winner: black) in 59 turns
Game 204/1000 (Winner: white) in 51 turns
Game 205/1000 (Winner: white) in 47 turns
Game 206/1000 (Winner: white) in 42 turns
Game 207/1000 (Winner: black) in 52 turns
Game 208/1000 (Winner: white) in 50 turns
Game 209/1000 (Winner: white) in 48 turns
Game 210/1000 (Winner: black) in 42 turns
Game 211/1000 (Winner: white) in 54 turns
Game 212/1000 (Winner: white) in 47 turns
Game 213/1000 (Winner: black) in 46 turns
Game 214/1000 (Winner: black) in 47 turns
Game 215/1000 (Winner: white) in 45 turns
Game 216/1000 (Winner: black) in 46 turns
Game 217/1000 (Winner: black) in 48 turns
Game 218/1000 (Winner: white) in 45 turns
Game 219/1000 (Winner: white) in 45 turns
Game 220/1000 (Winner: black) in 48 turns
Game 221/1000 (Winner: black) in 55 turns
Game 222/1000 (Winner: white) in 5

Game 396/1000 (Winner: white) in 45 turns
Game 397/1000 (Winner: white) in 53 turns
Game 398/1000 (Winner: white) in 46 turns
Game 399/1000 (Winner: black) in 48 turns
Game 400/1000 (Winner: black) in 51 turns
Game 401/1000 (Winner: black) in 48 turns
Game 402/1000 (Winner: black) in 48 turns
Game 403/1000 (Winner: white) in 50 turns
Game 404/1000 (Winner: black) in 50 turns
Game 405/1000 (Winner: white) in 47 turns
Game 406/1000 (Winner: white) in 50 turns
Game 407/1000 (Winner: black) in 50 turns
Game 408/1000 (Winner: white) in 45 turns
Game 409/1000 (Winner: white) in 51 turns
Game 410/1000 (Winner: white) in 45 turns
Game 411/1000 (Winner: black) in 43 turns
Game 412/1000 (Winner: white) in 45 turns
Game 413/1000 (Winner: black) in 51 turns
Game 414/1000 (Winner: white) in 49 turns
Game 415/1000 (Winner: black) in 45 turns
Game 416/1000 (Winner: white) in 44 turns
Game 417/1000 (Winner: white) in 37 turns
Game 418/1000 (Winner: white) in 44 turns
Game 419/1000 (Winner: black) in 4

Game 592/1000 (Winner: white) in 43 turns
Game 593/1000 (Winner: white) in 46 turns
Game 594/1000 (Winner: black) in 43 turns
Game 595/1000 (Winner: white) in 49 turns
Game 596/1000 (Winner: white) in 53 turns
Game 597/1000 (Winner: black) in 53 turns
Game 598/1000 (Winner: white) in 50 turns
Game 599/1000 (Winner: white) in 46 turns
Game 600/1000 (Winner: black) in 43 turns
Game 601/1000 (Winner: black) in 49 turns
Game 602/1000 (Winner: black) in 47 turns
Game 603/1000 (Winner: white) in 50 turns
Game 604/1000 (Winner: white) in 49 turns
Game 605/1000 (Winner: black) in 49 turns
Game 606/1000 (Winner: black) in 53 turns
Game 607/1000 (Winner: black) in 53 turns
Game 608/1000 (Winner: white) in 45 turns
Game 609/1000 (Winner: white) in 45 turns
Game 610/1000 (Winner: black) in 50 turns
Game 611/1000 (Winner: white) in 47 turns
Game 612/1000 (Winner: black) in 49 turns
Game 613/1000 (Winner: black) in 48 turns
Game 614/1000 (Winner: white) in 40 turns
Game 615/1000 (Winner: white) in 5

Game 789/1000 (Winner: white) in 42 turns
Game 790/1000 (Winner: white) in 50 turns
Game 791/1000 (Winner: black) in 45 turns
Game 792/1000 (Winner: white) in 43 turns
Game 793/1000 (Winner: black) in 55 turns
Game 794/1000 (Winner: white) in 43 turns
Game 795/1000 (Winner: black) in 50 turns
Game 796/1000 (Winner: white) in 44 turns
Game 797/1000 (Winner: white) in 52 turns
Game 798/1000 (Winner: white) in 43 turns
Game 799/1000 (Winner: white) in 51 turns
Game 800/1000 (Winner: black) in 51 turns
Game 801/1000 (Winner: black) in 31 turns
Game 802/1000 (Winner: black) in 49 turns
Game 803/1000 (Winner: black) in 42 turns
Game 804/1000 (Winner: black) in 42 turns
Game 805/1000 (Winner: white) in 52 turns
Game 806/1000 (Winner: white) in 47 turns
Game 807/1000 (Winner: black) in 43 turns
Game 808/1000 (Winner: black) in 43 turns
Game 809/1000 (Winner: black) in 48 turns
Game 810/1000 (Winner: white) in 46 turns
Game 811/1000 (Winner: white) in 51 turns
Game 812/1000 (Winner: white) in 4

Game 985/1000 (Winner: black) in 50 turns
Game 986/1000 (Winner: black) in 47 turns
Game 987/1000 (Winner: black) in 46 turns
Game 988/1000 (Winner: black) in 41 turns
Game 989/1000 (Winner: black) in 39 turns
Game 990/1000 (Winner: white) in 43 turns
Game 991/1000 (Winner: white) in 43 turns
Game 992/1000 (Winner: black) in 49 turns
Game 993/1000 (Winner: white) in 45 turns
Game 994/1000 (Winner: white) in 53 turns
Game 995/1000 (Winner: black) in 48 turns
Game 996/1000 (Winner: black) in 47 turns
Game 997/1000 (Winner: black) in 45 turns
Game 998/1000 (Winner: black) in 48 turns
Game 999/1000 (Winner: black) in 45 turns
[Game 0] TD-GammonPlayer (black) vs RandomPlayer (white) 0:1 of 1 games (0.00%)
[Game 1] TD-GammonPlayer (black) vs RandomPlayer (white) 1:1 of 2 games (50.00%)
[Game 2] TD-GammonPlayer (black) vs RandomPlayer (white) 2:1 of 3 games (66.67%)
[Game 3] TD-GammonPlayer (black) vs RandomPlayer (white) 2:2 of 4 games (50.00%)
[Game 4] TD-GammonPlayer (black) vs RandomPlaye

[Game 93] TD-GammonPlayer (black) vs RandomPlayer (white) 41:53 of 94 games (43.62%)
[Game 94] TD-GammonPlayer (black) vs RandomPlayer (white) 41:54 of 95 games (43.16%)
[Game 95] TD-GammonPlayer (black) vs RandomPlayer (white) 41:55 of 96 games (42.71%)
[Game 96] TD-GammonPlayer (black) vs RandomPlayer (white) 41:56 of 97 games (42.27%)
[Game 97] TD-GammonPlayer (black) vs RandomPlayer (white) 42:56 of 98 games (42.86%)
[Game 98] TD-GammonPlayer (black) vs RandomPlayer (white) 43:56 of 99 games (43.43%)
[Game 99] TD-GammonPlayer (black) vs RandomPlayer (white) 43:57 of 100 games (43.00%)


Nach 1000 Trainingsspielen kann man schon eine Verbesserung erkennen!

Also weiter trainieren, diesmal direkt 4000 Spiele!

In [51]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.train(4000,1000)

Restoring checkpoint: checkpoints/checkpoint.ckpt-51444
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-51444
Game 0/4000 (Winner: white) in 55 turns
Game 1/4000 (Winner: white) in 46 turns
Game 2/4000 (Winner: black) in 46 turns
Game 3/4000 (Winner: black) in 52 turns
Game 4/4000 (Winner: white) in 42 turns
Game 5/4000 (Winner: black) in 54 turns
Game 6/4000 (Winner: black) in 53 turns
Game 7/4000 (Winner: white) in 49 turns
Game 8/4000 (Winner: white) in 41 turns
Game 9/4000 (Winner: white) in 45 turns
Game 10/4000 (Winner: white) in 52 turns
Game 11/4000 (Winner: black) in 41 turns
Game 12/4000 (Winner: black) in 39 turns
Game 13/4000 (Winner: white) in 45 turns
Game 14/4000 (Winner: white) in 55 turns
Game 15/4000 (Winner: white) in 49 turns
Game 16/4000 (Winner: black) in 50 turns
Game 17/4000 (Winner: black) in 47 turns
Game 18/4000 (Winner: white) in 49 turns
Game 19/4000 (Winner: black) in 55 turns
Game 20/4000 (Winner: black) in 43 turns
Game 21/4000 (Win

Game 391/4000 (Winner: black) in 55 turns
Game 392/4000 (Winner: black) in 49 turns
Game 393/4000 (Winner: white) in 47 turns
Game 394/4000 (Winner: white) in 51 turns
Game 395/4000 (Winner: white) in 46 turns
Game 396/4000 (Winner: white) in 53 turns
Game 397/4000 (Winner: black) in 46 turns
Game 398/4000 (Winner: black) in 47 turns
Game 399/4000 (Winner: black) in 54 turns
Game 400/4000 (Winner: white) in 37 turns
Game 401/4000 (Winner: white) in 45 turns
Game 402/4000 (Winner: black) in 53 turns
Game 403/4000 (Winner: black) in 52 turns
Game 404/4000 (Winner: white) in 45 turns
Game 405/4000 (Winner: white) in 44 turns
Game 406/4000 (Winner: white) in 45 turns
Game 407/4000 (Winner: white) in 46 turns
Game 408/4000 (Winner: black) in 50 turns
Game 409/4000 (Winner: black) in 48 turns
Game 410/4000 (Winner: black) in 44 turns
Game 411/4000 (Winner: white) in 57 turns
Game 412/4000 (Winner: white) in 37 turns
Game 413/4000 (Winner: white) in 43 turns
Game 414/4000 (Winner: black) in 5

Game 587/4000 (Winner: white) in 46 turns
Game 588/4000 (Winner: white) in 55 turns
Game 589/4000 (Winner: black) in 53 turns
Game 590/4000 (Winner: black) in 54 turns
Game 591/4000 (Winner: white) in 52 turns
Game 592/4000 (Winner: white) in 50 turns
Game 593/4000 (Winner: black) in 41 turns
Game 594/4000 (Winner: black) in 45 turns
Game 595/4000 (Winner: white) in 42 turns
Game 596/4000 (Winner: white) in 45 turns
Game 597/4000 (Winner: white) in 43 turns
Game 598/4000 (Winner: white) in 43 turns
Game 599/4000 (Winner: white) in 48 turns
Game 600/4000 (Winner: white) in 50 turns
Game 601/4000 (Winner: white) in 41 turns
Game 602/4000 (Winner: white) in 43 turns
Game 603/4000 (Winner: white) in 51 turns
Game 604/4000 (Winner: black) in 53 turns
Game 605/4000 (Winner: white) in 49 turns
Game 606/4000 (Winner: black) in 51 turns
Game 607/4000 (Winner: white) in 48 turns
Game 608/4000 (Winner: white) in 45 turns
Game 609/4000 (Winner: black) in 53 turns
Game 610/4000 (Winner: white) in 4

Game 784/4000 (Winner: black) in 41 turns
Game 785/4000 (Winner: white) in 47 turns
Game 786/4000 (Winner: white) in 47 turns
Game 787/4000 (Winner: black) in 49 turns
Game 788/4000 (Winner: black) in 51 turns
Game 789/4000 (Winner: black) in 47 turns
Game 790/4000 (Winner: black) in 53 turns
Game 791/4000 (Winner: black) in 57 turns
Game 792/4000 (Winner: white) in 42 turns
Game 793/4000 (Winner: white) in 51 turns
Game 794/4000 (Winner: black) in 44 turns
Game 795/4000 (Winner: black) in 47 turns
Game 796/4000 (Winner: white) in 48 turns
Game 797/4000 (Winner: white) in 47 turns
Game 798/4000 (Winner: white) in 42 turns
Game 799/4000 (Winner: white) in 46 turns
Game 800/4000 (Winner: black) in 48 turns
Game 801/4000 (Winner: white) in 43 turns
Game 802/4000 (Winner: white) in 61 turns
Game 803/4000 (Winner: white) in 50 turns
Game 804/4000 (Winner: white) in 47 turns
Game 805/4000 (Winner: black) in 44 turns
Game 806/4000 (Winner: white) in 51 turns
Game 807/4000 (Winner: black) in 4

Game 980/4000 (Winner: black) in 44 turns
Game 981/4000 (Winner: white) in 45 turns
Game 982/4000 (Winner: black) in 41 turns
Game 983/4000 (Winner: white) in 47 turns
Game 984/4000 (Winner: white) in 50 turns
Game 985/4000 (Winner: white) in 48 turns
Game 986/4000 (Winner: white) in 43 turns
Game 987/4000 (Winner: black) in 50 turns
Game 988/4000 (Winner: black) in 49 turns
Game 989/4000 (Winner: white) in 50 turns
Game 990/4000 (Winner: white) in 43 turns
Game 991/4000 (Winner: black) in 47 turns
Game 992/4000 (Winner: white) in 48 turns
Game 993/4000 (Winner: white) in 43 turns
Game 994/4000 (Winner: white) in 46 turns
Game 995/4000 (Winner: white) in 51 turns
Game 996/4000 (Winner: white) in 45 turns
Game 997/4000 (Winner: black) in 45 turns
Game 998/4000 (Winner: white) in 47 turns
Game 999/4000 (Winner: black) in 52 turns
Progress saved!
[Game 0] TD-GammonPlayer (black) vs RandomPlayer (white) 1:0 of 1 games (100.00%)
[Game 1] TD-GammonPlayer (black) vs RandomPlayer (white) 1:1 o

...

Game 3258/4000 (Winner: white) in 37 turns
Game 3259/4000 (Winner: black) in 49 turns
Game 3260/4000 (Winner: black) in 51 turns
Game 3261/4000 (Winner: black) in 46 turns
Game 3262/4000 (Winner: white) in 54 turns
Game 3263/4000 (Winner: white) in 51 turns
Game 3264/4000 (Winner: white) in 56 turns
Game 3265/4000 (Winner: white) in 51 turns
Game 3266/4000 (Winner: white) in 49 turns
Game 3267/4000 (Winner: black) in 59 turns
Game 3268/4000 (Winner: white) in 50 turns
Game 3269/4000 (Winner: white) in 47 turns
Game 3270/4000 (Winner: white) in 37 turns
Game 3271/4000 (Winner: white) in 50 turns
Game 3272/4000 (Winner: white) in 58 turns
Game 3273/4000 (Winner: white) in 49 turns
Game 3274/4000 (Winner: black) in 49 turns
Game 3275/4000 (Winner: black) in 59 turns
Game 3276/4000 (Winner: white) in 47 turns
Game 3277/4000 (Winner: white) in 56 turns
Game 3278/4000 (Winner: white) in 54 turns
Game 3279/4000 (Winner: black) in 58 turns
Game 3280/4000 (Winner: black) in 46 turns
Game 3281/4

Game 3449/4000 (Winner: black) in 72 turns
Game 3450/4000 (Winner: white) in 66 turns
Game 3451/4000 (Winner: black) in 42 turns
Game 3452/4000 (Winner: white) in 69 turns
Game 3453/4000 (Winner: white) in 47 turns
Game 3454/4000 (Winner: black) in 68 turns
Game 3455/4000 (Winner: white) in 46 turns
Game 3456/4000 (Winner: white) in 85 turns
Game 3457/4000 (Winner: white) in 55 turns
Game 3458/4000 (Winner: white) in 64 turns
Game 3459/4000 (Winner: white) in 58 turns
Game 3460/4000 (Winner: black) in 68 turns
Game 3461/4000 (Winner: black) in 68 turns
Game 3462/4000 (Winner: white) in 88 turns
Game 3463/4000 (Winner: white) in 52 turns
Game 3464/4000 (Winner: white) in 66 turns
Game 3465/4000 (Winner: black) in 50 turns
Game 3466/4000 (Winner: white) in 49 turns
Game 3467/4000 (Winner: black) in 56 turns
Game 3468/4000 (Winner: black) in 52 turns
Game 3469/4000 (Winner: black) in 61 turns
Game 3470/4000 (Winner: white) in 74 turns
Game 3471/4000 (Winner: white) in 44 turns
Game 3472/4

Game 3641/4000 (Winner: white) in 62 turns
Game 3642/4000 (Winner: white) in 66 turns
Game 3643/4000 (Winner: black) in 68 turns
Game 3644/4000 (Winner: white) in 52 turns
Game 3645/4000 (Winner: white) in 62 turns
Game 3646/4000 (Winner: black) in 62 turns
Game 3647/4000 (Winner: white) in 70 turns
Game 3648/4000 (Winner: white) in 91 turns
Game 3649/4000 (Winner: black) in 49 turns
Game 3650/4000 (Winner: white) in 42 turns
Game 3651/4000 (Winner: white) in 66 turns
Game 3652/4000 (Winner: black) in 81 turns
Game 3653/4000 (Winner: black) in 49 turns
Game 3654/4000 (Winner: white) in 42 turns
Game 3655/4000 (Winner: black) in 76 turns
Game 3656/4000 (Winner: black) in 49 turns
Game 3657/4000 (Winner: white) in 59 turns
Game 3658/4000 (Winner: white) in 56 turns
Game 3659/4000 (Winner: black) in 57 turns
Game 3660/4000 (Winner: black) in 64 turns
Game 3661/4000 (Winner: black) in 60 turns
Game 3662/4000 (Winner: white) in 52 turns
Game 3663/4000 (Winner: white) in 59 turns
Game 3664/4

Game 3832/4000 (Winner: white) in 64 turns
Game 3833/4000 (Winner: black) in 65 turns
Game 3834/4000 (Winner: black) in 69 turns
Game 3835/4000 (Winner: black) in 65 turns
Game 3836/4000 (Winner: white) in 60 turns
Game 3837/4000 (Winner: black) in 57 turns
Game 3838/4000 (Winner: white) in 57 turns
Game 3839/4000 (Winner: black) in 62 turns
Game 3840/4000 (Winner: black) in 60 turns
Game 3841/4000 (Winner: black) in 63 turns
Game 3842/4000 (Winner: black) in 73 turns
Game 3843/4000 (Winner: black) in 84 turns
Game 3844/4000 (Winner: white) in 84 turns
Game 3845/4000 (Winner: black) in 41 turns
Game 3846/4000 (Winner: white) in 95 turns
Game 3847/4000 (Winner: white) in 80 turns
Game 3848/4000 (Winner: black) in 56 turns
Game 3849/4000 (Winner: white) in 63 turns
Game 3850/4000 (Winner: black) in 68 turns
Game 3851/4000 (Winner: white) in 64 turns
Game 3852/4000 (Winner: white) in 86 turns
Game 3853/4000 (Winner: black) in 69 turns
Game 3854/4000 (Winner: white) in 99 turns
Game 3855/4

[Game 13] TD-GammonPlayer (black) vs RandomPlayer (white) 14:0 of 14 games (100.00%)
[Game 14] TD-GammonPlayer (black) vs RandomPlayer (white) 15:0 of 15 games (100.00%)
[Game 15] TD-GammonPlayer (black) vs RandomPlayer (white) 16:0 of 16 games (100.00%)
[Game 16] TD-GammonPlayer (black) vs RandomPlayer (white) 17:0 of 17 games (100.00%)
[Game 17] TD-GammonPlayer (black) vs RandomPlayer (white) 18:0 of 18 games (100.00%)
[Game 18] TD-GammonPlayer (black) vs RandomPlayer (white) 19:0 of 19 games (100.00%)
[Game 19] TD-GammonPlayer (black) vs RandomPlayer (white) 20:0 of 20 games (100.00%)
[Game 20] TD-GammonPlayer (black) vs RandomPlayer (white) 21:0 of 21 games (100.00%)
[Game 21] TD-GammonPlayer (black) vs RandomPlayer (white) 22:0 of 22 games (100.00%)
[Game 22] TD-GammonPlayer (black) vs RandomPlayer (white) 23:0 of 23 games (100.00%)
[Game 23] TD-GammonPlayer (black) vs RandomPlayer (white) 24:0 of 24 games (100.00%)
[Game 24] TD-GammonPlayer (black) vs RandomPlayer (white) 25:0 of

### Trainingsfortschritt nach 5000 Spielen
Während TD-Gammon nach 4000 Spielen nur 50% gegen random gewinnt so gewinnt es nach 5000 Spielen zu 99% dagegen.

Wie schlägt sich TD-Gammon gegen andere Gegner?

In [3]:
import Player

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.test(games = 200, enemyPlayer = ValuePlayer(Game.PLAYERS[1], Player.singleton))

Restoring checkpoint: checkpoints/checkpoint.ckpt-258909
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-258909
[Game 0] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 0:1 of 1 games (0.00%)
[Game 1] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:1 of 2 games (50.00%)
[Game 2] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:2 of 3 games (33.33%)
[Game 3] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:3 of 4 games (25.00%)
[Game 4] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 2:3 of 5 games (40.00%)
[Game 5] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:3 of 6 games (50.00%)
[Game 6] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:4 of 7 games (42.86%)
[Game 7] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:5 of 8 games (37.50%)
[Game 8] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 4:5 of 9 games (44.44%)
[Game 9] TD-GammonPlayer (black) vs Val

[Game 87] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 56:32 of 88 games (63.64%)
[Game 88] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 56:33 of 89 games (62.92%)
[Game 89] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 56:34 of 90 games (62.22%)
[Game 90] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 57:34 of 91 games (62.64%)
[Game 91] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 58:34 of 92 games (63.04%)
[Game 92] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 59:34 of 93 games (63.44%)
[Game 93] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 59:35 of 94 games (62.77%)
[Game 94] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 60:35 of 95 games (63.16%)
[Game 95] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 60:36 of 96 games (62.50%)
[Game 96] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 61:36 of 97 games (62.89%)
[Game 97] TD-GammonPlayer (black) vs Val

[Game 171] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 109:63 of 172 games (63.37%)
[Game 172] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 110:63 of 173 games (63.58%)
[Game 173] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 111:63 of 174 games (63.79%)
[Game 174] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 111:64 of 175 games (63.43%)
[Game 175] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 112:64 of 176 games (63.64%)
[Game 176] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 113:64 of 177 games (63.84%)
[Game 177] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 113:65 of 178 games (63.48%)
[Game 178] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 114:65 of 179 games (63.69%)
[Game 179] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 115:65 of 180 games (63.89%)
[Game 180] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 116:65 of 181 games (64.09%)
[Game 181]

Nur 62% Siegesrate gegen Singleton? Da muss noch mehr trainiert werden!

In [4]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.train(5000,1000)

Restoring checkpoint: checkpoints/checkpoint.ckpt-258909
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-258909
Game 0/5000 (Winner: white) in 58 turns
Game 1/5000 (Winner: white) in 88 turns
Game 2/5000 (Winner: black) in 64 turns
Game 3/5000 (Winner: black) in 77 turns
Game 4/5000 (Winner: white) in 70 turns
Game 5/5000 (Winner: black) in 86 turns
Game 6/5000 (Winner: black) in 77 turns
Game 7/5000 (Winner: black) in 67 turns
Game 8/5000 (Winner: black) in 45 turns
Game 9/5000 (Winner: white) in 82 turns
Game 10/5000 (Winner: black) in 58 turns
Game 11/5000 (Winner: black) in 68 turns
Game 12/5000 (Winner: black) in 98 turns
Game 13/5000 (Winner: white) in 50 turns
Game 14/5000 (Winner: white) in 94 turns
Game 15/5000 (Winner: white) in 56 turns
Game 16/5000 (Winner: black) in 68 turns
Game 17/5000 (Winner: black) in 83 turns
Game 18/5000 (Winner: black) in 64 turns
Game 19/5000 (Winner: white) in 58 turns
Game 20/5000 (Winner: black) in 67 turns
Game 21/5000 (W

Game 195/5000 (Winner: black) in 59 turns
Game 196/5000 (Winner: black) in 85 turns
Game 197/5000 (Winner: white) in 70 turns
Game 198/5000 (Winner: white) in 67 turns
Game 199/5000 (Winner: black) in 106 turns
Game 200/5000 (Winner: white) in 53 turns
Game 201/5000 (Winner: black) in 89 turns
Game 202/5000 (Winner: black) in 72 turns
Game 203/5000 (Winner: black) in 61 turns
Game 204/5000 (Winner: white) in 93 turns
Game 205/5000 (Winner: black) in 58 turns
Game 206/5000 (Winner: white) in 74 turns
Game 207/5000 (Winner: black) in 85 turns
Game 208/5000 (Winner: black) in 67 turns
Game 209/5000 (Winner: white) in 49 turns
Game 210/5000 (Winner: white) in 135 turns
Game 211/5000 (Winner: white) in 67 turns
Game 212/5000 (Winner: black) in 84 turns
Game 213/5000 (Winner: white) in 74 turns
Game 214/5000 (Winner: white) in 74 turns
Game 215/5000 (Winner: black) in 57 turns
Game 216/5000 (Winner: white) in 51 turns
Game 217/5000 (Winner: black) in 70 turns
Game 218/5000 (Winner: white) in

Game 390/5000 (Winner: black) in 67 turns
Game 391/5000 (Winner: black) in 56 turns
Game 392/5000 (Winner: white) in 79 turns
Game 393/5000 (Winner: white) in 78 turns
Game 394/5000 (Winner: black) in 62 turns
Game 395/5000 (Winner: white) in 56 turns
Game 396/5000 (Winner: black) in 55 turns
Game 397/5000 (Winner: black) in 108 turns
Game 398/5000 (Winner: white) in 67 turns
Game 399/5000 (Winner: white) in 54 turns
Game 400/5000 (Winner: white) in 120 turns
Game 401/5000 (Winner: white) in 115 turns
Game 402/5000 (Winner: black) in 86 turns
Game 403/5000 (Winner: black) in 85 turns
Game 404/5000 (Winner: white) in 61 turns
Game 405/5000 (Winner: black) in 93 turns
Game 406/5000 (Winner: white) in 64 turns
Game 407/5000 (Winner: white) in 62 turns
Game 408/5000 (Winner: black) in 54 turns
Game 409/5000 (Winner: black) in 68 turns
Game 410/5000 (Winner: white) in 76 turns
Game 411/5000 (Winner: white) in 70 turns
Game 412/5000 (Winner: black) in 70 turns
Game 413/5000 (Winner: white) i

Game 585/5000 (Winner: white) in 51 turns
Game 586/5000 (Winner: white) in 90 turns
Game 587/5000 (Winner: black) in 64 turns
Game 588/5000 (Winner: white) in 87 turns
Game 589/5000 (Winner: black) in 63 turns
Game 590/5000 (Winner: white) in 81 turns
Game 591/5000 (Winner: white) in 62 turns
Game 592/5000 (Winner: black) in 102 turns
Game 593/5000 (Winner: black) in 104 turns
Game 594/5000 (Winner: black) in 72 turns
Game 595/5000 (Winner: black) in 75 turns
Game 596/5000 (Winner: white) in 71 turns
Game 597/5000 (Winner: black) in 75 turns
Game 598/5000 (Winner: black) in 107 turns
Game 599/5000 (Winner: white) in 79 turns
Game 600/5000 (Winner: white) in 116 turns
Game 601/5000 (Winner: white) in 65 turns
Game 602/5000 (Winner: white) in 65 turns
Game 603/5000 (Winner: black) in 53 turns
Game 604/5000 (Winner: black) in 70 turns
Game 605/5000 (Winner: white) in 112 turns
Game 606/5000 (Winner: white) in 65 turns
Game 607/5000 (Winner: white) in 62 turns
Game 608/5000 (Winner: white)

Game 779/5000 (Winner: black) in 92 turns
Game 780/5000 (Winner: black) in 84 turns
Game 781/5000 (Winner: black) in 115 turns
Game 782/5000 (Winner: black) in 94 turns
Game 783/5000 (Winner: black) in 103 turns
Game 784/5000 (Winner: white) in 91 turns
Game 785/5000 (Winner: black) in 53 turns
Game 786/5000 (Winner: white) in 71 turns
Game 787/5000 (Winner: white) in 43 turns
Game 788/5000 (Winner: white) in 52 turns
Game 789/5000 (Winner: black) in 65 turns
Game 790/5000 (Winner: white) in 58 turns
Game 791/5000 (Winner: black) in 109 turns
Game 792/5000 (Winner: white) in 92 turns
Game 793/5000 (Winner: white) in 108 turns
Game 794/5000 (Winner: white) in 104 turns
Game 795/5000 (Winner: black) in 101 turns
Game 796/5000 (Winner: black) in 79 turns
Game 797/5000 (Winner: white) in 66 turns
Game 798/5000 (Winner: black) in 91 turns
Game 799/5000 (Winner: black) in 123 turns
Game 800/5000 (Winner: white) in 49 turns
Game 801/5000 (Winner: black) in 69 turns
Game 802/5000 (Winner: blac

...

Game 4376/5000 (Winner: black) in 53 turns
Game 4377/5000 (Winner: black) in 114 turns
Game 4378/5000 (Winner: white) in 58 turns
Game 4379/5000 (Winner: white) in 103 turns
Game 4380/5000 (Winner: black) in 63 turns
Game 4381/5000 (Winner: white) in 43 turns
Game 4382/5000 (Winner: white) in 83 turns
Game 4383/5000 (Winner: black) in 72 turns
Game 4384/5000 (Winner: black) in 55 turns
Game 4385/5000 (Winner: black) in 180 turns
Game 4386/5000 (Winner: white) in 88 turns
Game 4387/5000 (Winner: white) in 91 turns
Game 4388/5000 (Winner: white) in 80 turns
Game 4389/5000 (Winner: white) in 85 turns
Game 4390/5000 (Winner: white) in 140 turns
Game 4391/5000 (Winner: black) in 127 turns
Game 4392/5000 (Winner: black) in 81 turns
Game 4393/5000 (Winner: black) in 54 turns
Game 4394/5000 (Winner: white) in 75 turns
Game 4395/5000 (Winner: white) in 43 turns
Game 4396/5000 (Winner: white) in 139 turns
Game 4397/5000 (Winner: black) in 111 turns
Game 4398/5000 (Winner: white) in 72 turns
Game

Game 4566/5000 (Winner: white) in 98 turns
Game 4567/5000 (Winner: white) in 76 turns
Game 4568/5000 (Winner: black) in 97 turns
Game 4569/5000 (Winner: white) in 150 turns
Game 4570/5000 (Winner: black) in 79 turns
Game 4571/5000 (Winner: black) in 53 turns
Game 4572/5000 (Winner: black) in 54 turns
Game 4573/5000 (Winner: white) in 68 turns
Game 4574/5000 (Winner: black) in 51 turns
Game 4575/5000 (Winner: black) in 64 turns
Game 4576/5000 (Winner: black) in 62 turns
Game 4577/5000 (Winner: white) in 109 turns
Game 4578/5000 (Winner: white) in 89 turns
Game 4579/5000 (Winner: white) in 87 turns
Game 4580/5000 (Winner: black) in 113 turns
Game 4581/5000 (Winner: white) in 69 turns
Game 4582/5000 (Winner: white) in 118 turns
Game 4583/5000 (Winner: white) in 90 turns
Game 4584/5000 (Winner: black) in 58 turns
Game 4585/5000 (Winner: white) in 161 turns
Game 4586/5000 (Winner: white) in 173 turns
Game 4587/5000 (Winner: black) in 137 turns
Game 4588/5000 (Winner: white) in 140 turns
Gam

Game 4756/5000 (Winner: white) in 89 turns
Game 4757/5000 (Winner: black) in 110 turns
Game 4758/5000 (Winner: black) in 59 turns
Game 4759/5000 (Winner: black) in 104 turns
Game 4760/5000 (Winner: white) in 53 turns
Game 4761/5000 (Winner: white) in 82 turns
Game 4762/5000 (Winner: black) in 72 turns
Game 4763/5000 (Winner: black) in 44 turns
Game 4764/5000 (Winner: white) in 83 turns
Game 4765/5000 (Winner: white) in 127 turns
Game 4766/5000 (Winner: white) in 136 turns
Game 4767/5000 (Winner: white) in 49 turns
Game 4768/5000 (Winner: white) in 57 turns
Game 4769/5000 (Winner: white) in 62 turns
Game 4770/5000 (Winner: black) in 74 turns
Game 4771/5000 (Winner: white) in 86 turns
Game 4772/5000 (Winner: white) in 64 turns
Game 4773/5000 (Winner: white) in 111 turns
Game 4774/5000 (Winner: white) in 54 turns
Game 4775/5000 (Winner: black) in 93 turns
Game 4776/5000 (Winner: white) in 82 turns
Game 4777/5000 (Winner: black) in 63 turns
Game 4778/5000 (Winner: white) in 48 turns
Game 4

Game 4946/5000 (Winner: black) in 78 turns
Game 4947/5000 (Winner: white) in 70 turns
Game 4948/5000 (Winner: black) in 73 turns
Game 4949/5000 (Winner: black) in 60 turns
Game 4950/5000 (Winner: black) in 131 turns
Game 4951/5000 (Winner: black) in 51 turns
Game 4952/5000 (Winner: black) in 68 turns
Game 4953/5000 (Winner: white) in 88 turns
Game 4954/5000 (Winner: white) in 89 turns
Game 4955/5000 (Winner: black) in 62 turns
Game 4956/5000 (Winner: white) in 90 turns
Game 4957/5000 (Winner: white) in 39 turns
Game 4958/5000 (Winner: black) in 52 turns
Game 4959/5000 (Winner: white) in 91 turns
Game 4960/5000 (Winner: black) in 104 turns
Game 4961/5000 (Winner: black) in 70 turns
Game 4962/5000 (Winner: black) in 61 turns
Game 4963/5000 (Winner: white) in 70 turns
Game 4964/5000 (Winner: black) in 114 turns
Game 4965/5000 (Winner: black) in 87 turns
Game 4966/5000 (Winner: black) in 81 turns
Game 4967/5000 (Winner: white) in 53 turns
Game 4968/5000 (Winner: white) in 130 turns
Game 49

[Game 71] TD-GammonPlayer (black) vs RandomPlayer (white) 72:0 of 72 games (100.00%)
[Game 72] TD-GammonPlayer (black) vs RandomPlayer (white) 73:0 of 73 games (100.00%)
[Game 73] TD-GammonPlayer (black) vs RandomPlayer (white) 74:0 of 74 games (100.00%)
[Game 74] TD-GammonPlayer (black) vs RandomPlayer (white) 75:0 of 75 games (100.00%)
[Game 75] TD-GammonPlayer (black) vs RandomPlayer (white) 76:0 of 76 games (100.00%)
[Game 76] TD-GammonPlayer (black) vs RandomPlayer (white) 77:0 of 77 games (100.00%)
[Game 77] TD-GammonPlayer (black) vs RandomPlayer (white) 78:0 of 78 games (100.00%)
[Game 78] TD-GammonPlayer (black) vs RandomPlayer (white) 79:0 of 79 games (100.00%)
[Game 79] TD-GammonPlayer (black) vs RandomPlayer (white) 80:0 of 80 games (100.00%)
[Game 80] TD-GammonPlayer (black) vs RandomPlayer (white) 81:0 of 81 games (100.00%)
[Game 81] TD-GammonPlayer (black) vs RandomPlayer (white) 82:0 of 82 games (100.00%)
[Game 82] TD-GammonPlayer (black) vs RandomPlayer (white) 83:0 of

### Trainingsfortschritt nach 10000 Spielen

Wenn man die Spiellänge vom Anfang des Trainings mit Ende des Trainings vergleicht fällt auf, dass nach einer gewissen Eingewöhnungsphase die Spiele in ca. 50 Zügen zu Ende waren und nach 5000 Spielen auf durchschnittlich 70 gestiegen ist. 

Während den nächsten 5000 Spielen erhöhen sich die Züge immer weiter. Nachdem das Spiel gelernt hat wie man gewinnt, lernt es nun wie man nicht so schnell verliert => Die Spiele werden länger, das Niveau steigt

##### TD-Gammon vs Singleton

In [5]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.test(games = 200, enemyPlayer = ValuePlayer(Game.PLAYERS[1], Player.singleton))

Restoring checkpoint: checkpoints/checkpoint.ckpt-688188
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-688188
[Game 0] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:0 of 1 games (100.00%)
[Game 1] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 2:0 of 2 games (100.00%)
[Game 2] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:0 of 3 games (100.00%)
[Game 3] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:1 of 4 games (75.00%)
[Game 4] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 4:1 of 5 games (80.00%)
[Game 5] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 5:1 of 6 games (83.33%)
[Game 6] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 6:1 of 7 games (85.71%)
[Game 7] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 7:1 of 8 games (87.50%)
[Game 8] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 8:1 of 9 games (88.89%)
[Game 9] TD-GammonPlayer (black) vs

[Game 85] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 73:13 of 86 games (84.88%)
[Game 86] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 74:13 of 87 games (85.06%)
[Game 87] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 75:13 of 88 games (85.23%)
[Game 88] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 76:13 of 89 games (85.39%)
[Game 89] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 77:13 of 90 games (85.56%)
[Game 90] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 78:13 of 91 games (85.71%)
[Game 91] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 78:14 of 92 games (84.78%)
[Game 92] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 79:14 of 93 games (84.95%)
[Game 93] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 80:14 of 94 games (85.11%)
[Game 94] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 81:14 of 95 games (85.26%)
[Game 95] TD-GammonPlayer (black) vs Val

[Game 170] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 141:30 of 171 games (82.46%)
[Game 171] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 142:30 of 172 games (82.56%)
[Game 172] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 143:30 of 173 games (82.66%)
[Game 173] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 144:30 of 174 games (82.76%)
[Game 174] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 145:30 of 175 games (82.86%)
[Game 175] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 146:30 of 176 games (82.95%)
[Game 176] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 147:30 of 177 games (83.05%)
[Game 177] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 148:30 of 178 games (83.15%)
[Game 178] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 148:31 of 179 games (82.68%)
[Game 179] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 149:31 of 180 games (82.78%)
[Game 180]

Von 62% auf 82%. Deutliche Steigerung, TD-Gammon lernt immer noch!

Was passiert wenn wir noch mal 10000 Spiele trainieren lassen?

In [6]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.train(10000,1000)

Restoring checkpoint: checkpoints/checkpoint.ckpt-688188
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-688188
Game 0/10000 (Winner: black) in 68 turns
Game 1/10000 (Winner: white) in 62 turns
Game 2/10000 (Winner: white) in 100 turns
Game 3/10000 (Winner: black) in 67 turns
Game 4/10000 (Winner: black) in 141 turns
Game 5/10000 (Winner: black) in 102 turns
Game 6/10000 (Winner: white) in 103 turns
Game 7/10000 (Winner: white) in 61 turns
Game 8/10000 (Winner: white) in 81 turns
Game 9/10000 (Winner: white) in 159 turns
Game 10/10000 (Winner: white) in 80 turns
Game 11/10000 (Winner: white) in 105 turns
Game 12/10000 (Winner: black) in 86 turns
Game 13/10000 (Winner: white) in 67 turns
Game 14/10000 (Winner: black) in 67 turns
Game 15/10000 (Winner: black) in 66 turns
Game 16/10000 (Winner: black) in 134 turns
Game 17/10000 (Winner: white) in 63 turns
Game 18/10000 (Winner: black) in 64 turns
Game 19/10000 (Winner: black) in 77 turns
Game 20/10000 (Winner: black)

Game 189/10000 (Winner: white) in 84 turns
Game 190/10000 (Winner: white) in 80 turns
Game 191/10000 (Winner: black) in 125 turns
Game 192/10000 (Winner: white) in 116 turns
Game 193/10000 (Winner: black) in 81 turns
Game 194/10000 (Winner: black) in 139 turns
Game 195/10000 (Winner: black) in 58 turns
Game 196/10000 (Winner: white) in 93 turns
Game 197/10000 (Winner: white) in 107 turns
Game 198/10000 (Winner: black) in 74 turns
Game 199/10000 (Winner: white) in 96 turns
Game 200/10000 (Winner: black) in 67 turns
Game 201/10000 (Winner: black) in 106 turns
Game 202/10000 (Winner: black) in 131 turns
Game 203/10000 (Winner: white) in 107 turns
Game 204/10000 (Winner: white) in 95 turns
Game 205/10000 (Winner: black) in 62 turns
Game 206/10000 (Winner: white) in 74 turns
Game 207/10000 (Winner: white) in 83 turns
Game 208/10000 (Winner: black) in 71 turns
Game 209/10000 (Winner: black) in 121 turns
Game 210/10000 (Winner: white) in 79 turns
Game 211/10000 (Winner: black) in 106 turns
Ga

Game 378/10000 (Winner: black) in 100 turns
Game 379/10000 (Winner: black) in 78 turns
Game 380/10000 (Winner: black) in 132 turns
Game 381/10000 (Winner: black) in 66 turns
Game 382/10000 (Winner: black) in 127 turns
Game 383/10000 (Winner: black) in 100 turns
Game 384/10000 (Winner: white) in 218 turns
Game 385/10000 (Winner: black) in 83 turns
Game 386/10000 (Winner: white) in 67 turns
Game 387/10000 (Winner: black) in 122 turns
Game 388/10000 (Winner: white) in 138 turns
Game 389/10000 (Winner: black) in 80 turns
Game 390/10000 (Winner: black) in 91 turns
Game 391/10000 (Winner: black) in 86 turns
Game 392/10000 (Winner: black) in 116 turns
Game 393/10000 (Winner: white) in 73 turns
Game 394/10000 (Winner: white) in 80 turns
Game 395/10000 (Winner: black) in 130 turns
Game 396/10000 (Winner: black) in 83 turns
Game 397/10000 (Winner: black) in 104 turns
Game 398/10000 (Winner: white) in 61 turns
Game 399/10000 (Winner: white) in 45 turns
Game 400/10000 (Winner: black) in 101 turns


Game 568/10000 (Winner: black) in 86 turns
Game 569/10000 (Winner: white) in 58 turns
Game 570/10000 (Winner: white) in 79 turns
Game 571/10000 (Winner: white) in 78 turns
Game 572/10000 (Winner: black) in 82 turns
Game 573/10000 (Winner: black) in 68 turns
Game 574/10000 (Winner: white) in 48 turns
Game 575/10000 (Winner: black) in 74 turns
Game 576/10000 (Winner: black) in 71 turns
Game 577/10000 (Winner: black) in 84 turns
Game 578/10000 (Winner: black) in 72 turns
Game 579/10000 (Winner: black) in 69 turns
Game 580/10000 (Winner: white) in 59 turns
Game 581/10000 (Winner: black) in 81 turns
Game 582/10000 (Winner: black) in 118 turns
Game 583/10000 (Winner: white) in 91 turns
Game 584/10000 (Winner: white) in 72 turns
Game 585/10000 (Winner: white) in 89 turns
Game 586/10000 (Winner: white) in 103 turns
Game 587/10000 (Winner: black) in 65 turns
Game 588/10000 (Winner: white) in 49 turns
Game 589/10000 (Winner: white) in 146 turns
Game 590/10000 (Winner: white) in 60 turns
Game 591

Game 758/10000 (Winner: white) in 77 turns
Game 759/10000 (Winner: white) in 91 turns
Game 760/10000 (Winner: white) in 65 turns
Game 761/10000 (Winner: white) in 71 turns
Game 762/10000 (Winner: black) in 64 turns
Game 763/10000 (Winner: white) in 108 turns
Game 764/10000 (Winner: black) in 74 turns
Game 765/10000 (Winner: white) in 92 turns
Game 766/10000 (Winner: white) in 141 turns
Game 767/10000 (Winner: black) in 98 turns
Game 768/10000 (Winner: black) in 77 turns
Game 769/10000 (Winner: white) in 45 turns
Game 770/10000 (Winner: black) in 111 turns
Game 771/10000 (Winner: white) in 139 turns
Game 772/10000 (Winner: black) in 119 turns
Game 773/10000 (Winner: black) in 150 turns
Game 774/10000 (Winner: black) in 54 turns
Game 775/10000 (Winner: black) in 46 turns
Game 776/10000 (Winner: black) in 67 turns
Game 777/10000 (Winner: black) in 64 turns
Game 778/10000 (Winner: black) in 68 turns
Game 779/10000 (Winner: white) in 67 turns
Game 780/10000 (Winner: black) in 74 turns
Game 

...

Game 9425/10000 (Winner: black) in 127 turns
Game 9426/10000 (Winner: white) in 121 turns
Game 9427/10000 (Winner: black) in 84 turns
Game 9428/10000 (Winner: black) in 106 turns
Game 9429/10000 (Winner: black) in 43 turns
Game 9430/10000 (Winner: white) in 85 turns
Game 9431/10000 (Winner: black) in 71 turns
Game 9432/10000 (Winner: black) in 90 turns
Game 9433/10000 (Winner: black) in 82 turns
Game 9434/10000 (Winner: black) in 72 turns
Game 9435/10000 (Winner: black) in 94 turns
Game 9436/10000 (Winner: black) in 144 turns
Game 9437/10000 (Winner: black) in 149 turns
Game 9438/10000 (Winner: white) in 71 turns
Game 9439/10000 (Winner: white) in 104 turns
Game 9440/10000 (Winner: black) in 144 turns
Game 9441/10000 (Winner: black) in 83 turns
Game 9442/10000 (Winner: black) in 123 turns
Game 9443/10000 (Winner: black) in 60 turns
Game 9444/10000 (Winner: black) in 93 turns
Game 9445/10000 (Winner: white) in 81 turns
Game 9446/10000 (Winner: white) in 88 turns
Game 9447/10000 (Winner:

Game 9611/10000 (Winner: white) in 88 turns
Game 9612/10000 (Winner: white) in 91 turns
Game 9613/10000 (Winner: black) in 72 turns
Game 9614/10000 (Winner: black) in 66 turns
Game 9615/10000 (Winner: white) in 207 turns
Game 9616/10000 (Winner: black) in 110 turns
Game 9617/10000 (Winner: black) in 62 turns
Game 9618/10000 (Winner: white) in 71 turns
Game 9619/10000 (Winner: black) in 97 turns
Game 9620/10000 (Winner: black) in 134 turns
Game 9621/10000 (Winner: black) in 93 turns
Game 9622/10000 (Winner: black) in 105 turns
Game 9623/10000 (Winner: black) in 67 turns
Game 9624/10000 (Winner: black) in 43 turns
Game 9625/10000 (Winner: white) in 62 turns
Game 9626/10000 (Winner: black) in 87 turns
Game 9627/10000 (Winner: black) in 78 turns
Game 9628/10000 (Winner: white) in 121 turns
Game 9629/10000 (Winner: black) in 61 turns
Game 9630/10000 (Winner: black) in 78 turns
Game 9631/10000 (Winner: white) in 98 turns
Game 9632/10000 (Winner: black) in 72 turns
Game 9633/10000 (Winner: wh

Game 9796/10000 (Winner: black) in 92 turns
Game 9797/10000 (Winner: black) in 89 turns
Game 9798/10000 (Winner: black) in 131 turns
Game 9799/10000 (Winner: black) in 95 turns
Game 9800/10000 (Winner: black) in 155 turns
Game 9801/10000 (Winner: black) in 82 turns
Game 9802/10000 (Winner: black) in 74 turns
Game 9803/10000 (Winner: white) in 94 turns
Game 9804/10000 (Winner: white) in 64 turns
Game 9805/10000 (Winner: black) in 185 turns
Game 9806/10000 (Winner: black) in 48 turns
Game 9807/10000 (Winner: black) in 74 turns
Game 9808/10000 (Winner: white) in 97 turns
Game 9809/10000 (Winner: white) in 146 turns
Game 9810/10000 (Winner: white) in 51 turns
Game 9811/10000 (Winner: white) in 56 turns
Game 9812/10000 (Winner: white) in 73 turns
Game 9813/10000 (Winner: black) in 203 turns
Game 9814/10000 (Winner: black) in 50 turns
Game 9815/10000 (Winner: black) in 71 turns
Game 9816/10000 (Winner: black) in 124 turns
Game 9817/10000 (Winner: black) in 114 turns
Game 9818/10000 (Winner: 

Game 9981/10000 (Winner: white) in 72 turns
Game 9982/10000 (Winner: white) in 78 turns
Game 9983/10000 (Winner: black) in 59 turns
Game 9984/10000 (Winner: black) in 89 turns
Game 9985/10000 (Winner: black) in 72 turns
Game 9986/10000 (Winner: black) in 50 turns
Game 9987/10000 (Winner: black) in 93 turns
Game 9988/10000 (Winner: black) in 85 turns
Game 9989/10000 (Winner: black) in 114 turns
Game 9990/10000 (Winner: black) in 76 turns
Game 9991/10000 (Winner: white) in 141 turns
Game 9992/10000 (Winner: white) in 132 turns
Game 9993/10000 (Winner: black) in 81 turns
Game 9994/10000 (Winner: black) in 87 turns
Game 9995/10000 (Winner: white) in 162 turns
Game 9996/10000 (Winner: white) in 89 turns
Game 9997/10000 (Winner: white) in 78 turns
Game 9998/10000 (Winner: black) in 64 turns
Game 9999/10000 (Winner: black) in 68 turns
[Game 0] TD-GammonPlayer (black) vs RandomPlayer (white) 1:0 of 1 games (100.00%)
[Game 1] TD-GammonPlayer (black) vs RandomPlayer (white) 2:0 of 2 games (100.0

[Game 87] TD-GammonPlayer (black) vs RandomPlayer (white) 88:0 of 88 games (100.00%)
[Game 88] TD-GammonPlayer (black) vs RandomPlayer (white) 89:0 of 89 games (100.00%)
[Game 89] TD-GammonPlayer (black) vs RandomPlayer (white) 90:0 of 90 games (100.00%)
[Game 90] TD-GammonPlayer (black) vs RandomPlayer (white) 91:0 of 91 games (100.00%)
[Game 91] TD-GammonPlayer (black) vs RandomPlayer (white) 92:0 of 92 games (100.00%)
[Game 92] TD-GammonPlayer (black) vs RandomPlayer (white) 93:0 of 93 games (100.00%)
[Game 93] TD-GammonPlayer (black) vs RandomPlayer (white) 94:0 of 94 games (100.00%)
[Game 94] TD-GammonPlayer (black) vs RandomPlayer (white) 95:0 of 95 games (100.00%)
[Game 95] TD-GammonPlayer (black) vs RandomPlayer (white) 96:0 of 96 games (100.00%)
[Game 96] TD-GammonPlayer (black) vs RandomPlayer (white) 97:0 of 97 games (100.00%)
[Game 97] TD-GammonPlayer (black) vs RandomPlayer (white) 98:0 of 98 games (100.00%)
[Game 98] TD-GammonPlayer (black) vs RandomPlayer (white) 99:0 of

##### TD-Gammon vs Singleton

In [8]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.test(games = 200, enemyPlayer = ValuePlayer(Game.PLAYERS[1], Player.singleton))

Restoring checkpoint: checkpoints/checkpoint.ckpt-1617757
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-1617757
[Game 0] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:0 of 1 games (100.00%)
[Game 1] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 1:1 of 2 games (50.00%)
[Game 2] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 2:1 of 3 games (66.67%)
[Game 3] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:1 of 4 games (75.00%)
[Game 4] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 3:2 of 5 games (60.00%)
[Game 5] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 4:2 of 6 games (66.67%)
[Game 6] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 5:2 of 7 games (71.43%)
[Game 7] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 6:2 of 8 games (75.00%)
[Game 8] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 7:2 of 9 games (77.78%)
[Game 9] TD-GammonPlayer (black) vs

[Game 85] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 70:16 of 86 games (81.40%)
[Game 86] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 71:16 of 87 games (81.61%)
[Game 87] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 72:16 of 88 games (81.82%)
[Game 88] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 72:17 of 89 games (80.90%)
[Game 89] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 73:17 of 90 games (81.11%)
[Game 90] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 74:17 of 91 games (81.32%)
[Game 91] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 75:17 of 92 games (81.52%)
[Game 92] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 76:17 of 93 games (81.72%)
[Game 93] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 77:17 of 94 games (81.91%)
[Game 94] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 78:17 of 95 games (82.11%)
[Game 95] TD-GammonPlayer (black) vs Val

[Game 169] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 140:30 of 170 games (82.35%)
[Game 170] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 141:30 of 171 games (82.46%)
[Game 171] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 142:30 of 172 games (82.56%)
[Game 172] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 143:30 of 173 games (82.66%)
[Game 173] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 144:30 of 174 games (82.76%)
[Game 174] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 145:30 of 175 games (82.86%)
[Game 175] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 146:30 of 176 games (82.95%)
[Game 176] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 146:31 of 177 games (82.49%)
[Game 177] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 147:31 of 178 games (82.58%)
[Game 178] TD-GammonPlayer (black) vs ValuePlayer [singleton] (white) 148:31 of 179 games (82.68%)
[Game 179]

Nach weiteren 10000 weiteren Spielen keine Steigerung, schade.

Aber __83%__ zeigt dennoch eine deutliche Überlegenheit

In [4]:
import Player

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.test(games = 200, enemyPlayer = ValuePlayer(Game.PLAYERS[1], Player.single_to_go))

Restoring checkpoint: checkpoints/checkpoint.ckpt-1617757
INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt-1617757
[Game 0] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 1:0 of 1 games (100.00%)
[Game 1] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 1:1 of 2 games (50.00%)
[Game 2] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 2:1 of 3 games (66.67%)
[Game 3] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 3:1 of 4 games (75.00%)
[Game 4] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 4:1 of 5 games (80.00%)
[Game 5] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 5:1 of 6 games (83.33%)
[Game 6] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 6:1 of 7 games (85.71%)
[Game 7] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 7:1 of 8 games (87.50%)
[Game 8] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 8:1 of 9 games (88.89%)
[Game 9]

[Game 83] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 66:18 of 84 games (78.57%)
[Game 84] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 67:18 of 85 games (78.82%)
[Game 85] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 67:19 of 86 games (77.91%)
[Game 86] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 68:19 of 87 games (78.16%)
[Game 87] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 69:19 of 88 games (78.41%)
[Game 88] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 70:19 of 89 games (78.65%)
[Game 89] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 71:19 of 90 games (78.89%)
[Game 90] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 71:20 of 91 games (78.02%)
[Game 91] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 72:20 of 92 games (78.26%)
[Game 92] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 72:21 of 93 games (77.42%)
[Game 93] 

[Game 165] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 119:47 of 166 games (71.69%)
[Game 166] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 120:47 of 167 games (71.86%)
[Game 167] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 121:47 of 168 games (72.02%)
[Game 168] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 122:47 of 169 games (72.19%)
[Game 169] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 122:48 of 170 games (71.76%)
[Game 170] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 122:49 of 171 games (71.35%)
[Game 171] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 123:49 of 172 games (71.51%)
[Game 172] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 124:49 of 173 games (71.68%)
[Game 173] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 125:49 of 174 games (71.84%)
[Game 174] TD-GammonPlayer (black) vs ValuePlayer [single_to_go] (white) 126:49 of

71% Sieg gegen SingleToGo!

In [1]:
import tensorflow as tf
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=False)
    model.train(20000,1000)

Game 0/20000 (Winner: black) in 131 turns
Game 1/20000 (Winner: black) in 121 turns
Game 2/20000 (Winner: black) in 300 turns
Game 3/20000 (Winner: black) in 181 turns
Game 4/20000 (Winner: black) in 162 turns
Game 5/20000 (Winner: black) in 387 turns
Game 6/20000 (Winner: black) in 599 turns
Game 7/20000 (Winner: black) in 196 turns
Game 8/20000 (Winner: black) in 131 turns
Game 9/20000 (Winner: black) in 140 turns
Game 10/20000 (Winner: black) in 83 turns
Game 11/20000 (Winner: black) in 277 turns
Game 12/20000 (Winner: black) in 109 turns
Game 13/20000 (Winner: black) in 191 turns
Game 14/20000 (Winner: white) in 502 turns
Game 15/20000 (Winner: white) in 103 turns
Game 16/20000 (Winner: white) in 193 turns
Game 17/20000 (Winner: black) in 337 turns
Game 18/20000 (Winner: white) in 54 turns
Game 19/20000 (Winner: white) in 123 turns
Game 20/20000 (Winner: black) in 82 turns
Game 21/20000 (Winner: black) in 100 turns
Game 22/20000 (Winner: black) in 147 turns
Game 23/20000 (Winner: b

Game 193/20000 (Winner: black) in 47 turns
Game 194/20000 (Winner: white) in 42 turns
Game 195/20000 (Winner: white) in 54 turns
Game 196/20000 (Winner: white) in 70 turns
Game 197/20000 (Winner: black) in 53 turns
Game 198/20000 (Winner: white) in 48 turns
Game 199/20000 (Winner: black) in 50 turns
Game 200/20000 (Winner: white) in 40 turns
Game 201/20000 (Winner: white) in 51 turns
Game 202/20000 (Winner: black) in 52 turns
Game 203/20000 (Winner: white) in 58 turns
Game 204/20000 (Winner: black) in 45 turns
Game 205/20000 (Winner: white) in 48 turns
Game 206/20000 (Winner: black) in 50 turns
Game 207/20000 (Winner: black) in 42 turns
Game 208/20000 (Winner: black) in 43 turns
Game 209/20000 (Winner: white) in 53 turns
Game 210/20000 (Winner: white) in 53 turns
Game 211/20000 (Winner: black) in 44 turns
Game 212/20000 (Winner: white) in 49 turns
Game 213/20000 (Winner: black) in 52 turns
Game 214/20000 (Winner: black) in 49 turns
Game 215/20000 (Winner: white) in 54 turns
Game 216/20

Game 383/20000 (Winner: black) in 44 turns
Game 384/20000 (Winner: white) in 47 turns
Game 385/20000 (Winner: black) in 53 turns
Game 386/20000 (Winner: white) in 41 turns
Game 387/20000 (Winner: white) in 46 turns
Game 388/20000 (Winner: white) in 58 turns
Game 389/20000 (Winner: black) in 37 turns
Game 390/20000 (Winner: black) in 47 turns
Game 391/20000 (Winner: white) in 42 turns
Game 392/20000 (Winner: white) in 48 turns
Game 393/20000 (Winner: white) in 45 turns
Game 394/20000 (Winner: white) in 41 turns
Game 395/20000 (Winner: black) in 43 turns
Game 396/20000 (Winner: white) in 44 turns
Game 397/20000 (Winner: black) in 41 turns
Game 398/20000 (Winner: black) in 54 turns
Game 399/20000 (Winner: white) in 44 turns
Game 400/20000 (Winner: black) in 43 turns
Game 401/20000 (Winner: black) in 44 turns
Game 402/20000 (Winner: black) in 45 turns
Game 403/20000 (Winner: white) in 49 turns
Game 404/20000 (Winner: white) in 55 turns
Game 405/20000 (Winner: black) in 43 turns
Game 406/20

Game 575/20000 (Winner: black) in 53 turns
Game 576/20000 (Winner: black) in 53 turns
Game 577/20000 (Winner: white) in 54 turns
Game 578/20000 (Winner: black) in 45 turns
Game 579/20000 (Winner: black) in 45 turns
Game 580/20000 (Winner: black) in 46 turns
Game 581/20000 (Winner: black) in 51 turns
Game 582/20000 (Winner: white) in 45 turns
Game 583/20000 (Winner: black) in 49 turns
Game 584/20000 (Winner: black) in 54 turns
Game 585/20000 (Winner: black) in 53 turns
Game 586/20000 (Winner: white) in 45 turns
Game 587/20000 (Winner: black) in 54 turns
Game 588/20000 (Winner: white) in 52 turns
Game 589/20000 (Winner: white) in 54 turns
Game 590/20000 (Winner: white) in 50 turns
Game 591/20000 (Winner: black) in 52 turns
Game 592/20000 (Winner: white) in 51 turns
Game 593/20000 (Winner: white) in 59 turns
Game 594/20000 (Winner: white) in 53 turns
Game 595/20000 (Winner: black) in 56 turns
Game 596/20000 (Winner: black) in 45 turns
Game 597/20000 (Winner: white) in 48 turns
Game 598/20

Game 766/20000 (Winner: white) in 53 turns
Game 767/20000 (Winner: white) in 45 turns
Game 768/20000 (Winner: black) in 43 turns
Game 769/20000 (Winner: white) in 40 turns
Game 770/20000 (Winner: black) in 51 turns
Game 771/20000 (Winner: white) in 59 turns
Game 772/20000 (Winner: white) in 59 turns
Game 773/20000 (Winner: white) in 47 turns
Game 774/20000 (Winner: black) in 40 turns
Game 775/20000 (Winner: black) in 49 turns
Game 776/20000 (Winner: black) in 46 turns
Game 777/20000 (Winner: black) in 48 turns
Game 778/20000 (Winner: white) in 42 turns
Game 779/20000 (Winner: white) in 45 turns
Game 780/20000 (Winner: white) in 50 turns
Game 781/20000 (Winner: black) in 50 turns
Game 782/20000 (Winner: black) in 47 turns
Game 783/20000 (Winner: black) in 48 turns
Game 784/20000 (Winner: white) in 55 turns
Game 785/20000 (Winner: white) in 47 turns
Game 786/20000 (Winner: white) in 52 turns
Game 787/20000 (Winner: black) in 54 turns
Game 788/20000 (Winner: black) in 47 turns
Game 789/20

...

Game 19459/20000 (Winner: black) in 103 turns
Game 19460/20000 (Winner: black) in 61 turns
Game 19461/20000 (Winner: black) in 183 turns
Game 19462/20000 (Winner: black) in 71 turns
Game 19463/20000 (Winner: black) in 159 turns
Game 19464/20000 (Winner: black) in 97 turns
Game 19465/20000 (Winner: black) in 84 turns
Game 19466/20000 (Winner: white) in 66 turns
Game 19467/20000 (Winner: white) in 62 turns
Game 19468/20000 (Winner: white) in 80 turns
Game 19469/20000 (Winner: black) in 51 turns
Game 19470/20000 (Winner: black) in 75 turns
Game 19471/20000 (Winner: black) in 101 turns
Game 19472/20000 (Winner: black) in 55 turns
Game 19473/20000 (Winner: white) in 51 turns
Game 19474/20000 (Winner: black) in 82 turns
Game 19475/20000 (Winner: black) in 135 turns
Game 19476/20000 (Winner: white) in 93 turns
Game 19477/20000 (Winner: white) in 54 turns
Game 19478/20000 (Winner: white) in 55 turns
Game 19479/20000 (Winner: white) in 115 turns
Game 19480/20000 (Winner: black) in 109 turns
Gam

Game 19640/20000 (Winner: black) in 125 turns
Game 19641/20000 (Winner: white) in 71 turns
Game 19642/20000 (Winner: white) in 45 turns
Game 19643/20000 (Winner: white) in 128 turns
Game 19644/20000 (Winner: white) in 89 turns
Game 19645/20000 (Winner: black) in 60 turns
Game 19646/20000 (Winner: white) in 62 turns
Game 19647/20000 (Winner: black) in 105 turns
Game 19648/20000 (Winner: black) in 76 turns
Game 19649/20000 (Winner: white) in 134 turns
Game 19650/20000 (Winner: black) in 103 turns
Game 19651/20000 (Winner: black) in 85 turns
Game 19652/20000 (Winner: black) in 97 turns
Game 19653/20000 (Winner: black) in 158 turns
Game 19654/20000 (Winner: black) in 162 turns
Game 19655/20000 (Winner: black) in 106 turns
Game 19656/20000 (Winner: white) in 79 turns
Game 19657/20000 (Winner: black) in 60 turns
Game 19658/20000 (Winner: white) in 68 turns
Game 19659/20000 (Winner: white) in 72 turns
Game 19660/20000 (Winner: white) in 87 turns
Game 19661/20000 (Winner: white) in 151 turns
G

Game 19821/20000 (Winner: white) in 51 turns
Game 19822/20000 (Winner: white) in 135 turns
Game 19823/20000 (Winner: white) in 86 turns
Game 19824/20000 (Winner: black) in 63 turns
Game 19825/20000 (Winner: black) in 77 turns
Game 19826/20000 (Winner: black) in 68 turns
Game 19827/20000 (Winner: black) in 123 turns
Game 19828/20000 (Winner: black) in 152 turns
Game 19829/20000 (Winner: white) in 72 turns
Game 19830/20000 (Winner: black) in 192 turns
Game 19831/20000 (Winner: white) in 63 turns
Game 19832/20000 (Winner: white) in 73 turns
Game 19833/20000 (Winner: white) in 86 turns
Game 19834/20000 (Winner: black) in 61 turns
Game 19835/20000 (Winner: black) in 47 turns
Game 19836/20000 (Winner: white) in 114 turns
Game 19837/20000 (Winner: white) in 130 turns
Game 19838/20000 (Winner: white) in 61 turns
Game 19839/20000 (Winner: black) in 44 turns
Game 19840/20000 (Winner: black) in 86 turns
Game 19841/20000 (Winner: black) in 166 turns
Game 19842/20000 (Winner: white) in 78 turns
Gam

[Game 1] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 2:0 of 2 games (100.00%)
[Game 2] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 3:0 of 3 games (100.00%)
[Game 3] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 4:0 of 4 games (100.00%)
[Game 4] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 5:0 of 5 games (100.00%)
[Game 5] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 6:0 of 6 games (100.00%)
[Game 6] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 7:0 of 7 games (100.00%)
[Game 7] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 8:0 of 8 games (100.00%)
[Game 8] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 9:0 of 9 games (100.00%)
[Game 9] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 10:0 of 10 games (100.00%)
[Game 10] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 11:0 of 11 games (100.00%)
[Game 11] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 12:0 of 12 games (100.00%)
[G

[Game 89] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 90:0 of 90 games (100.00%)
[Game 90] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 91:0 of 91 games (100.00%)
[Game 91] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 92:0 of 92 games (100.00%)
[Game 92] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 93:0 of 93 games (100.00%)
[Game 93] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 94:0 of 94 games (100.00%)
[Game 94] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 95:0 of 95 games (100.00%)
[Game 95] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 96:0 of 96 games (100.00%)
[Game 96] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 97:0 of 97 games (100.00%)
[Game 97] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 98:0 of 98 games (100.00%)
[Game 98] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 99:0 of 99 games (100.00%)
[Game 99] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 100:

In [2]:
import time
import tensorflow as tf
import Player
from FasterBackgammon import Game
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    start = time.time()
    model.test(games = 100, enemyPlayer = Player.ModelPlayer(Game.PLAYERS[1], model))
    print(time.time() - start, "Sekunden")

Restoring checkpoint: checkpoints/TD-Gammon/checkpoint.ckpt-1593683
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon/checkpoint.ckpt-1593683
[Game 0] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 0:2 of 2 games (0.00%)
[Game 2] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 1:2 of 3 games (33.33%)
[Game 3] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 2:2 of 4 games (50.00%)
[Game 4] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 2:3 of 5 games (40.00%)
[Game 5] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 2:4 of 6 games (33.33%)
[Game 6] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 3:4 of 7 games (42.86%)
[Game 7] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 3:5 of 8 games (37.50%)
[Game 8] ModelPlayer [TD-Gammon] (black) vs Mo

[Game 78] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 27:52 of 79 games (34.18%)
[Game 79] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 27:53 of 80 games (33.75%)
[Game 80] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 27:54 of 81 games (33.33%)
[Game 81] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 28:54 of 82 games (34.15%)
[Game 82] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 28:55 of 83 games (33.73%)
[Game 83] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 29:55 of 84 games (34.52%)
[Game 84] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 29:56 of 85 games (34.12%)
[Game 85] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 30:56 of 86 games (34.88%)
[Game 86] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gammon] (white) 30:57 of 87 games (34.48%)
[Game 87] ModelPlayer [TD-Gammon] (black) vs ModelPlayer [TD-Gam

## Veränderung der Gewichte

Vergleich zwischen dem untrainierten Modell und dem Modell nach 20000 Spielen

In [5]:
import tensorflow as tf
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=False)
    model.print_weights_biases()

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    model.print_weights_biases()

Input weights (198, 40) :
 [[-0.03571459 -0.037579   -0.01031089 ... -0.13093604  0.13955279
   0.0595663 ]
 [ 0.13237743  0.01403354  0.1507182  ...  0.028317   -0.09799042
  -0.02875666]
 [ 0.09765325  0.15176813 -0.04149317 ...  0.02669112 -0.15128882
   0.04669276]
 ...
 [-0.11672882 -0.1563137   0.10049219 ...  0.09624778 -0.13977727
   0.15105195]
 [-0.08005346  0.0949093  -0.05572678 ... -0.0049213   0.01480007
  -0.07911177]
 [-0.15780012  0.1460859  -0.05964663 ...  0.0670101  -0.05649975
  -0.10303518]]
Hidden bias (40,) :
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Hidden weights (40, 1) :
 [[ 0.03762424]
 [-0.08852616]
 [-0.01645768]
 [-0.23351958]
 [ 0.14016125]
 [-0.00587797]
 [ 0.09672761]
 [ 0.27749714]
 [-0.2361962 ]
 [-0.23592943]
 [-0.20349053]
 [ 0.07505777]
 [ 0.33222046]
 [-0.3665194 ]
 [ 0.07935932]
 [-0.15260246]
 [-0.16548486]
 [ 0.2458398 ]
 [-0.36454615]
 [ 0.06294778]
 [ 0.041730